#Task 3

In [1]:
# Install SWIG, a tool used to connect C/C++ code with Python. It's often used in RL environments to enable efficient communication between Python and low-level implementations of algorithms
!pip install -q swig

# Install the gym library with the box2d environment, used for 2D physics-based simulation tasks
!pip install -q gym[box2d]

# Install stable-baselines3 with extra dependencies (needed for various environments and features in the library), a set of RL algorithms implemented in PyTorch
!pip install stable-baselines3[extra]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 37.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 50.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidi

In [2]:
# Import the gymnasium library as gym, which provides various environments for developing and testing RL algorithms
import gymnasium as gym
import numpy as np

In [3]:
# Import PPO (Proximal Policy Optimization) algorithm from stable-baselines3. PPO is a policy-gradient method
from stable_baselines3 import PPO

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Import the MlpPolicy (Multi-Layer Perceptron) policy class. MlpPolicy is a policy class that uses MLP for function approximation
from stable_baselines3.dqn import MlpPolicy

# Import make_vec_env utility function to create vectorized environments for parallel execution of multiple environment instances. This can speed up training by allowing multiple agents to interact with their environments simultaneously
from stable_baselines3.common.env_util import make_vec_env

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [125]:
# Parallel Environments: make_vec_env is used to create multiple instances of the environment to be run in parallel. This allows the agent to collect more experience in less time, leading to faster training
vec_env = make_vec_env(
    "MountainCar-v0",          # The name of the environment to create (MountainCar-v0 in this case, from OpenAI's Gym)
    n_envs=16                  # Number of parallel environments to create
)

In [126]:
# Create the PPO model with the MLP policy. PPO is a RL algorithm known for its stability and efficiency
model = PPO("MlpPolicy", vec_env, verbose=1, n_steps=256) # Initialize the PPO algorithm with the MLP policy (policy network will be MLP, i.e. the NN will consist of fully connected layers), using the vectorized environment created above, set verbosity to 1 to print basic information and set the number of steps to run for each environment per update to 256

Using cpu device


In [127]:
# Train the model
model.learn(total_timesteps=1000000) # Train the PPO model on the environment for 1000000 timesteps

Streaming output truncated to the last 5000 lines.
|    value_loss           | 10.9        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 200         |
|    ep_rew_mean          | -200        |
| time/                   |             |
|    fps                  | 1368        |
|    iterations           | 8           |
|    time_elapsed         | 23          |
|    total_timesteps      | 32768       |
| train/                  |             |
|    approx_kl            | 0.011987487 |
|    clip_fraction        | 0.0299      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.07       |
|    explained_variance   | 0.00386     |
|    learning_rate        | 0.0003      |
|    loss                 | 0.869       |
|    n_updates            | 70          |
|    policy_gradient_loss | -0.00143    |
|    value_loss           | 7.01        |
-------------------------

In [128]:
# Import the evaluate_policy function from stable_baselines3 for evaluating the agent's performance
from stable_baselines3.common.evaluation import evaluate_policy

# Use a separate environement for evaluation (for more details see above)
eval_env = make_vec_env("MountainCar-v0", n_envs=1)

mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5) # Evaluate the agent's policy over 5 episodes in the eval_env. The function returns the mean reward and the standard deviation of rewards

# Print a summary of the agent's performance during the evaluation episodes
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward:-101.40 +/- 13.87


In [129]:
# Save the current trained model to a file named ppo_mc.zip. This is useful for preserving the model so it can be loaded and used later without having to retrain it
model.save("ppo_mc")

# Remove the model from memory. It is included here to demonstrate that the model can be completely removed from memory and then reloaded from the saved file
del model

# Load the model from from the previously saved file ppo_mc.zip. After this, the model object will be restored and can be used as before
model = PPO.load("ppo_mc")

In [130]:
# For visualization
from gym.wrappers.monitoring import video_recorder  # Import video recording utility from gym
from IPython.display import HTML  # Import HTML display utility from IPython
from IPython import display  # Import display utility from IPython
import glob  # Import glob for file pattern matching
import base64, io, os, shutil  # Import base64 for encoding, io for file handling, os for operating system interactions, and shutil for file operations
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv  # Import vectorized video recorder and dummy vectorized environment from stable-baselines3

# Set SDL video driver to 'dummy' to avoid issues on headless servers (servers without a graphical interface)
os.environ['SDL_VIDEODRIVER'] = 'dummy'

In [131]:
# Remove existing 'video' directory and create a new one to store video files
shutil.rmtree('video', ignore_errors=True)
os.makedirs("video", exist_ok=True)

# Function to display the recorded video. It searches for mp4 files in the 'video' directory, reads and encodes the video in base64 format, and displays it using an HTML video tag
def show_video():
    mp4list = glob.glob('video/*.mp4')  # Get list of mp4 files in the 'video' directory
    if len(mp4list) > 0:
        mp4 = mp4list[0]  # Get the first mp4 file from the list
        video = io.open(mp4, 'r+b').read()  # Read the video file
        encoded = base64.b64encode(video)  # Encode the video in base64
        display.display(HTML(data='''<video alt="test" autoplay loop controls style="height: 400px;">
              <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))  # Display the video in an HTML video tag
    else:
        print("Could not find video")  # Print error message if no video found


# Function to record a video of a RL model's performance in the MountainCar-v0 environment. It sets up the environment and video recorder, runs the model for a specified number of steps, and records the video
def show_video_of_model():
    """
    :param env_id: (str) environment ID
    :param model: (RL model) reinforcement learning model
    :param video_length: (int) length of the video in frames
    :param prefix: (str) prefix for the video file name
    :param video_folder: (str) folder to save the video
    """
    video_length = 1000  # Set video length to 1000 frames
    eval_env = make_vec_env("MountainCar-v0", n_envs=1)  # Create a vectorized environment for MountainCar-v0

    # Start the video at step=0 and record 1000 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder="video/",  # Folder to save the video
        record_video_trigger=lambda step: step == 0,  # Trigger video recording at the first step
        video_length=video_length,  # Length of the video
        name_prefix="",  # Prefix for the video file name
    )

    obs = eval_env.reset()  # Reset the environment to start
    for _ in range(video_length):
        action, _ = model.predict(obs)  # Predict the action using the model
        obs, _, _, _ = eval_env.step(action)  # Take the action in the environment and get the new observation

    # Close the video recorder
    eval_env.close()

In [132]:
show_video_of_model()

Saving video to /content/video/-step-0-to-step-1000.mp4
Moviepy - Building video /content/video/-step-0-to-step-1000.mp4.
Moviepy - Writing video /content/video/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /content/video/-step-0-to-step-1000.mp4


In [133]:
show_video()